In [1]:
import torch

In [2]:
print(torch.cuda.get_device_name())

NVIDIA A100-SXM4-40GB MIG 1g.5gb


### DataLoader and Dataset
* `Dataset` class of PyTorch is used to load custom datasets ,
* `DataLoader` class of PyTorch is used to make iterable batches of the dataset

In [3]:
dataset_link = "https://github.com/prashant-kikani/breast-cancer-detection/raw/master/breast-cancer-data.csv"

In [4]:
import pandas as pd
df = pd.read_csv(dataset_link)

In [5]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [6]:
df.drop(columns=['id','Unnamed: 32'], inplace=True)

In [7]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [9]:

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0], test_size=0.2)

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


In [11]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [12]:
X_train_tensor = torch.from_numpy(X_train).float()
X_test_tensor = torch.from_numpy(X_test).float()
y_train_tensor = torch.from_numpy(y_train).float()
y_test_tensor = torch.from_numpy(y_test).float()

### Dataset class has mainly three functions that we need to define to load the data:
* `def __init__()` which defines features and labels of dataset
* `__len__` whcih defines no of rows in dataset
* `__getitem__` returns no of features and label upon passing a specific index

In [13]:
from torch.utils.data import Dataset, DataLoader

In [14]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [33]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [34]:
len(dataset)

455

In [35]:
dataset[0]

(tensor([-0.5080, -1.1564, -0.5076, -0.5168, -1.0981, -0.4722, -0.2322, -0.5834,
         -0.5988, -0.1999, -0.4625, -0.6888, -0.3421, -0.4114, -0.8931,  0.2789,
          0.3725, -0.1231, -0.1498,  0.2894, -0.6109, -0.9921, -0.5446, -0.5813,
         -1.2998, -0.0476,  0.1905, -0.3169, -0.3499,  0.2347]),
 tensor(0.))

In [36]:
import os
print(os.cpu_count())
train_dataloader = DataLoader(train_dataset,
                        batch_size=100,
                        shuffle=True,
                        num_workers = os.cpu_count())
test_dataloader = DataLoader(test_dataset,
                        batch_size=100,
                        shuffle=False,
                        num_workers = os.cpu_count())

## We can make this process of making batches more faster by using parallelisation ...It has been integrated by using
# num_workers parameter in DataLoader class of PyTorch...it depends upon the hardware


256
